# Generate TimeSeries H5 from Library Profile to import in Web GUI


TVB has the custom notion of "profile". Based on the selected TVB profile, we enable/disable some TVB modules and default features (storage, load defaults).

We make a clear distinction between running TVB under WEB_PROFILE or LIBRARY_PROFILE.

Currently it is not possible to switch in the same code from one TVB profile to another, but it is  possible to share data, as described in the current demo. 

You can find more info about TVB profiles here: http://docs.thevirtualbrain.org/manuals/UserGuide/UserGuide-Shell.html#tvb-profiles

## Step 1

Let us start by running a standard TVB simulation, with the LIBRARY_PROFILE 
(from tvb.simulator.lab import * will set the LIBRARY_PROFILE).

In [1]:
%matplotlib widget
from tvb.simulator.lab import *
from tvb.datatypes import time_series
from tvb.basic.config import settings
import numpy as np

In [ ]:
jrm = models.JansenRit(mu=np.array([0.]), v0=np.array([6.]))
monitor = monitors.TemporalAverage(period=2 ** -2)

# the other aspects of the simulator are standard
sim = simulator.Simulator(
    model=jrm,
    connectivity=connectivity.Connectivity.from_file(),
    coupling=coupling.SigmoidalJansenRit(a=np.array([10.0])),
    monitors=(monitor,),
    simulation_length=1e3,
).configure()

# run it
(time_array, data_array), = sim.run()

## Step 2

write the simulation result in a H5 file.

In [3]:
from tvb.core.neocom import h5
from tvb.basic.profile import TvbProfile

TvbProfile.set_profile(TvbProfile.COMMAND_PROFILE)

series_of_time = time_series.TimeSeries(data=data_array, time=time_array, sample_period=monitor.period)
h5.store_complete_to_dir(series_of_time, ".")

<DataType(None, b5c085cf60874c67b64b7662a688f904, TimeSeriesIndex, tvb.adapters.datatypes.db.time_series, , , None, , , , , )>

Now you should have a file named "TimeSeriesRegion.......h5" in the current folder where ipython notebook has been launched. 

You can take this "TimeSeriesRegion.h5" file and import it into TVB web GUI, as described here:
http://docs.thevirtualbrain.org/manuals/UserGuide/UserGuide-UI_Project.html#data-structure

After import in TVB web GUI, you will have a new TimeSeriesRegion file in your current project, which can be used with TVB web visualizers.

